In [1]:
import pandas as pd

In [3]:
# bom_df = pd.read_excel('1182051_bom.XLSX')[['Item Number', 'Component number', 'Object description', 'Assembly indicator']]
# bom_df.to_csv('1182051_bom.csv', index=False)
bom_df = pd.read_csv('1182051_bom.csv')

In [6]:
# Step 1: Reset and group based on NaNs again (first group starts before first NaN)
bom_df['group'] = bom_df['Item Number'].isna().cumsum()

# Step 2: Prepare first group (top-level assembly) and set top-level parent
first_group = bom_df[bom_df['group'] == 0].copy()
parent_tree = {"1182051": {"Components": []}}

# Helper: Find alternates within a group (by duplicate Item Number)
def get_alternates_by_item(df_group):
    item_counts = df_group['Item Number'].value_counts()
    alt_item_numbers = item_counts[item_counts > 1].index
    alternates = {}
    for item_no in alt_item_numbers:
        items = df_group[df_group['Item Number'] == item_no]['Component number'].tolist()
        if len(items) > 1:
            alternates[item_no] = items
    return alternates

# Helper: Get next group for a component
def get_group_after_component(component_num):
    idx = bom_df[bom_df['Component number'] == component_num].index
    if len(idx) == 0:
        return pd.DataFrame()
    idx = idx[0]
    subsequent = bom_df[bom_df.index > idx]
    if subsequent.empty:
        return pd.DataFrame()
    group_id = subsequent.iloc[0]['group']
    return bom_df[bom_df['group'] == group_id].copy()

# Step 3: Process the first group
alt_by_item = get_alternates_by_item(first_group)
components_info = []

for _, row in first_group.iterrows():
    comp = row['Component number']
    item_no = row['Item Number']
    desc = row['Object description']
    is_assembly = str(row['Assembly indicator']).strip().lower() == 'yes'
    
    entry = {
        "Component": comp,
        "Item Number": item_no,
        "Description": desc,
        "Assembly": is_assembly,
        "Alternates": [],
        "Subcomponents": []
    }

    # If this item number has alternates
    if item_no in alt_by_item and comp in alt_by_item[item_no]:
        entry["Alternates"] = [c for c in alt_by_item[item_no] if c != comp]

    # If this is an assembly and contains 'Option' in description → its alternates are in the next group
    if is_assembly and 'option' in str(desc).lower():
        subgroup = get_group_after_component(comp)
        if not subgroup.empty:
            alt_group_by_item = get_alternates_by_item(subgroup)
            for _, subrow in subgroup.iterrows():
                subcomp = subrow['Component number']
                subitem_no = subrow['Item Number']
                if subitem_no in alt_group_by_item:
                    entry["Subcomponents"].append({
                        "Component": subcomp,
                        "Item Number": subitem_no,
                        "Alternates": alt_group_by_item[subitem_no]
                    })

    components_info.append(entry)

# Final structure
parent_tree["1182051"]["Components"] = components_info
parent_tree


{'1182051': {'Components': [{'Component': '1136907-01',
    'Item Number': 10.0,
    'Description': 'Assy, Diorite B1 FullRock 48GB, 1x200GbE',
    'Assembly': True,
    'Alternates': [],
    'Subcomponents': []},
   {'Component': '1107157-01',
    'Item Number': 20.0,
    'Description': 'Cable, HDMI, Standard Type A Male-to- St',
    'Assembly': False,
    'Alternates': [],
    'Subcomponents': []},
   {'Component': '1098823-01',
    'Item Number': 30.0,
    'Description': 'Cable, Power, Diorite to Dedicated Conn,',
    'Assembly': False,
    'Alternates': [],
    'Subcomponents': []},
   {'Component': '1112593-02',
    'Item Number': 40.0,
    'Description': 'ASM, Tray Cover, Air Baffle, Astoria, EV',
    'Assembly': False,
    'Alternates': ['1112593-02-FLG'],
    'Subcomponents': []},
   {'Component': '1112593-02-FLG',
    'Item Number': 40.0,
    'Description': 'ASM, Tray Cover, Air Baffle, Astoria(FLE',
    'Assembly': False,
    'Alternates': ['1112593-02'],
    'Subcomponents':

In [7]:
# Flatten the hierarchical structure into rows for a DataFrame
flattened_rows = []

parent = "1182051"
for comp in parent_tree[parent]["Components"]:
    base_row = {
        "Parent": parent,
        "Component": comp["Component"],
        "Item Number": comp["Item Number"],
        "Description": comp["Description"],
        "Assembly": comp["Assembly"],
        "Level": "Main",
        "Alternate Of": None
    }
    flattened_rows.append(base_row)
    
    # Add alternates
    for alt in comp["Alternates"]:
        flattened_rows.append({
            "Parent": parent,
            "Component": alt,
            "Item Number": comp["Item Number"],
            "Description": None,
            "Assembly": False,
            "Level": "Alternate",
            "Alternate Of": comp["Component"]
        })
    
    # Add subcomponents
    for sub in comp.get("Subcomponents", []):
        flattened_rows.append({
            "Parent": comp["Component"],
            "Component": sub["Component"],
            "Item Number": sub["Item Number"],
            "Description": None,
            "Assembly": False,
            "Level": "Subcomponent",
            "Alternate Of": None
        })
        # Add alternates of subcomponent
        for alt in sub["Alternates"]:
            if alt != sub["Component"]:
                flattened_rows.append({
                    "Parent": comp["Component"],
                    "Component": alt,
                    "Item Number": sub["Item Number"],
                    "Description": None,
                    "Assembly": False,
                    "Level": "Sub-Alternate",
                    "Alternate Of": sub["Component"]
                })

# Convert to DataFrame
result_df = pd.DataFrame(flattened_rows)
result_df.head(20)


,Parent,Component,Item Number,Description,Assembly,Level,Alternate Of
0,1182051,1136907-01,10.0,"Assy, Diorite B1 FullRock 48GB, 1x200GbE",True,Main,None
1,1182051,1107157-01,20.0,"Cable, HDMI, Standard Type A Male-to- St",False,Main,None
2,1182051,1098823-01,30.0,"Cable, Power, Diorite to Dedicated Conn,",False,Main,None
3,1182051,1112593-02,40.0,"ASM, Tray Cover, Air Baffle, Astoria, EV",False,Main,None
4,1182051,1112593-02-FLG,40.0,None,False,Alternate,1112593-02
5,1182051,1112593-02-FLG,40.0,"ASM, Tray Cover, Air Baffle, Astoria(FLE",False,Main,None
6,1182051,1112593-02,40.0,None,False,Alternate,1112593-02-FLG
7,1182051,1134400,50.0,"Assy, x16x16 IO Module",False,Main,None
8,1182051,1134400-01,50.0,None,False,Alternate,1134400
9,1182051,1134400-01-FGZ,50.0,None,False,Alternate,1134400


In [8]:
result_df[:50]

,Parent,Component,Item Number,Description,Assembly,Level,Alternate Of
0,1182051,1136907-01,10.0,"Assy, Diorite B1 FullRock 48GB, 1x200GbE",True,Main,None
1,1182051,1107157-01,20.0,"Cable, HDMI, Standard Type A Male-to- St",False,Main,None
2,1182051,1098823-01,30.0,"Cable, Power, Diorite to Dedicated Conn,",False,Main,None
3,1182051,1112593-02,40.0,"ASM, Tray Cover, Air Baffle, Astoria, EV",False,Main,None
4,1182051,1112593-02-FLG,40.0,None,False,Alternate,1112593-02
5,1182051,1112593-02-FLG,40.0,"ASM, Tray Cover, Air Baffle, Astoria(FLE",False,Main,None
6,1182051,1112593-02,40.0,None,False,Alternate,1112593-02-FLG
7,1182051,1134400,50.0,"Assy, x16x16 IO Module",False,Main,None
8,1182051,1134400-01,50.0,None,False,Alternate,1134400
9,1182051,1134400-01-FGZ,50.0,None,False,Alternate,1134400


In [2]:
import utils.helper_function as hf
query = '''
                    SELECT model_id, model_desc
                    FROM manufacturing_workorderdetail wo
                    WHERE workorder_id = '000007005634' and required_qty != 0
                    '''
df = hf.db_connect(query)

ModuleNotFoundError: No module named 'utils'

In [ ]:
impo